In [10]:
# GROUP1  

#name: Zeqing Yan/ Elda ferruku / Isaac hwang/ Sahil Ram


In [11]:
import pymysql
import pandas as pd
# import sqlalchemy

# Open Connection

In [12]:
connection = pymysql.connect(
	host='localhost',
	user='root',
	password= '',
    cursorclass=pymysql.cursors.DictCursor   
)

cursor = connection.cursor()


OperationalError: (1049, "Unknown database 'project_cis'")

In [ ]:
#Create a database to run and store tables

cursor.execute("SHOW DATABASES")
result = cursor.fetchall()
# print(cursor.fetchall()) #Verify that mydatabase appears in the list when print statement is run

In [ ]:
print(result)

In [ ]:
# the database we created named project_cis

In [ ]:
connection.close()

#  Extraction of Data from API's and CSV's

In [ ]:
collision_data = pd.read_csv("https://data.cityofnewyork.us/resource/h9gi-nx95.csv")
features = ["collision_id", "crash_time", "borough", "zip_code", "contributing_factor_vehicle_1", "contributing_factor_vehicle_2", "number_of_persons_injured", "number_of_persons_killed"]
collision_data = collision_data.loc[:,features]


In [ ]:
collision_data

In [ ]:
uber_data = pd.read_csv("./uber_nyc_enriched.csv")  
features = ["pickup_dt","pickups", "spd", "vsb", "temp", "pcp01", "pcp06", "pcp24", "sd", "borough", "hday"]
uber_data = uber_data.loc[:,features]

uber_data.insert(0, "uber_id", [i for i in range(len(uber_data))])

In [ ]:
uber_data

In [ ]:
noise_incident_data = pd.read_csv("./bar_locations.csv")  
features = ["Incident Zip", "Borough", "Location Type", "num_calls"]
noise_incident_data = noise_incident_data.loc[:,features]

noise_incident_data.insert(0, "incident_id", [i for i in range(len(noise_incident_data))])

In [ ]:
noise_incident_data

## Creating the fact dataframe

In [ ]:
facts = pd.data(data=uber_data['uber_id'])
facts.insert(1,'collision_id',collision_data['collision_id'])
facts.insert(2,'incident_id',collision_data['incident_id'])
facts

In [1]:
# Transformation of Data

In [ ]:
#remove NaN values
collision_data.dropna(inplace=True)

#change crash_time values into datetime format
collision_data['crash_time'] = pd.to_datetime(collision_data['crash_time'], format = '%H:%M').dt.time

#change data types
collision_data.dtypes
collision_data['collision_id'] = collision_data['collision_id'].astype(int)
collision_data['crash_time'] = collision_data['crash_time'].astype(str)
collision_data['borough'] = collision_data['borough'].astype(str)
collision_data['zip_code'] = collision_data['zip_code'].astype(int)
collision_data['contributing_factor_vehicle_1'] = collision_data['contributing_factor_vehicle_1'].astype(str)
collision_data['contributing_factor_vehicle_2'] = collision_data['contributing_factor_vehicle_2'].astype(str)
collision_data['number_of_persons_injured'] = collision_data['number_of_persons_injured'].astype(int)
collision_data['number_of_persons_killed'] = collision_data['number_of_persons_killed'].astype(int)

collision_data

In [ ]:
collision_data.columns

In [ ]:
#remove unnecessary value in borough
noise_incident_data.Borough.unique()
noise_incident_data = noise_incident_data[noise_incident_data.Borough != 'Unspecified']

#change data types
noise_incident_data.dtypes
noise_incident_data['incident_id'] = noise_incident_data['incident_id'].astype(int)
noise_incident_data['Incident Zip'] = noise_incident_data['Incident Zip'].astype(int)
noise_incident_data['Borough'] = noise_incident_data['Borough'].astype(str)
noise_incident_data['Location Type'] = noise_incident_data['Location Type'].astype(str)
noise_incident_data['num_calls'] = noise_incident_data['num_calls'].astype(int)



In [ ]:
noise_incident_data

In [ ]:
#split datetime into separate columns
uber_data['dates'] = pd.to_datetime(uber_data['pickup_dt'], format = '%Y/%m/%d %H:%M').dt.date
uber_data['time'] = pd.to_datetime(uber_data['pickup_dt'], format = '%Y/%m/%d %H:%M').dt.time
del uber_data['pickup_dt']

#remove NaN values
uber_data.dropna(inplace=True)

#remove unnecessary boroughs
uber_data.borough.unique()
uber_data = uber_data[uber_data.borough != 'EWR']

#match borough text with other dataframes
uber_data['borough'] = uber_data['borough'].str.upper()

#change data types
uber_data.dtypes
uber_data['uber_id'] = uber_data['uber_id'].astype(int)
uber_data['pickups'] = uber_data['pickups'].astype(int)
uber_data['borough'] = uber_data['borough'].astype(str)
uber_data['hday'] = uber_data['hday'].astype(str)
uber_data['dates'] = uber_data['dates'].astype(str)
uber_data['time'] = uber_data['time'].astype(str)



In [ ]:
uber_data

# Data load and Create table

In [ ]:
connection = pymysql.connect(
	host='localhost',
	user='root',
	password= ' !',
    db = 'project_cis',
    cursorclass=pymysql.cursors.DictCursor
    
)

cursor = connection.cursor()


In [ ]:
cursor.execute("SHOW TABLES")
result = cursor.fetchall()

In [ ]:
print(result)

In [ ]:
cursor.execute( "CREATE TABLE noise_incident_data (incident_id INT, Incident_Zip INT,Borough VARCHAR(255),Location_Type VARCHAR(255), num_calls INT)"
                 )

In [ ]:
cursor.execute("SHOW TABLES")
result = cursor.fetchall()

In [ ]:
print(result)

In [ ]:
cursor.execute("CREATE TABLE uber_data (uber_id INT, pickups INT, spd float, vsb float,temp float, pcp01 float, pcp06 float,pcp24 float, sd float, borough VARCHAR(255), hday VARCHAR(10), dates DATE, time TIME)") 

In [ ]:
cursor.execute("SHOW TABLES")
result = cursor.fetchall()
print(result)

In [ ]:
uber_data.dtypes

In [ ]:
noise_incident_data.dtypes

# Insert Values to MYSQL TABLE DATABASE 

In [ ]:
connection = pymysql.connect(
	host='localhost',
	user='root',
	password= 'password',
    db = 'project_cis',
    cursorclass=pymysql.cursors.DictCursor
    
)

cursor = connection.cursor()

In [ ]:
#sql = """ INSERT INTO noise_incident_data (incident_id,Incident_Zip,Borough,Location_Type,num_calls) VALUES %s,%s,%s,%s,%s"""

In [ ]:
df = uber_data

In [ ]:
engine = sqlalchemy.create_engine('mysql+pymysql://root:PASSWORD @localhost:3306/project_cis')

In [ ]:
df = pd.DataFrame(noise_incident_data)

df.to_sql(
    name = 'noise_incident_data',
    con = engine,
    index = False,
    if_exists = 'replace'
)

In [ ]:
df2 = pd.DataFrame(uber_data)

df2.to_sql(
    name = 'uber_data',
    con = engine,
    index = False,
    if_exists = 'replace'
)

In [ ]:
df3 = pd.DataFrame(collision_data)

df3.to_sql(
    name = 'collision_data',
    con = engine,
    index = False,
    if_exists = 'replace'
)